In [3]:
# import gc
#
# del train_loader  # Удаляем переменную
# gc.collect()  # Принудительная очистка памяти

In [4]:
import psutil
print(psutil.virtual_memory())  # Покажет загрузку RAM

svmem(total=17058336768, available=5480017920, percent=67.9, used=11578318848, free=5480017920)


In [6]:
!nvidia-smi -h # Покажет загрузку VRAM

ERROR: Option # is not valid for this command. Please run 'nvidia-smi -h' for help.



In [1]:
import matplotlib.pyplot as plt
import numpy as np

import torch
torch.cuda.empty_cache()
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, TensorDataset

from PIL import Image

import h5py
import glob

In [2]:
# import os
# import shutil
# import pandas as pd
# from tqdm import tqdm
#
# csv_file = "human_poses_data/train_answers.csv"
# train_image_dir = "human_poses_data/img_train"
# test_image_dir = "human_poses_data/img_test"
# output_dir = "data"
# train = "train"
# test = "test"
#
# src_train_path = os.path.join(output_dir, train)
# src_test_path = os.path.join(output_dir, test)
#
# os.makedirs(src_train_path, exist_ok=True)
# os.makedirs(src_test_path, exist_ok=True)
#
# df = pd.read_csv(csv_file)
#
# for _, row in tqdm(df.iterrows()):
#     image_id = str(row["img_id"]) + ".jpg"
#     class_name = str(row["target_feature"])
#
#     src_path = os.path.join(train_image_dir, image_id)
#
#     class_dir = os.path.join(src_train_path, class_name)
#     dst_path = os.path.join(class_dir, image_id)
#
#     os.makedirs(class_dir, exist_ok=True)
#
#     if os.path.exists(src_path):
#         shutil.copy(src_path, dst_path)
#     else:
#         print(f"Файл {src_path} не найден!")
#
# print("Копирование завершена!")


In [3]:
# test_image_dir = "human_poses_data/img_test"
# output_dir = "data"
# test = "test"
#
# src_test_path = os.path.join(output_dir, test)
# os.makedirs(src_test_path, exist_ok=True)
#
# if os.path.exists(test_image_dir):
#     for file_name in os.listdir(test_image_dir):
#         src_file = os.path.join(test_image_dir, file_name)
#         dst_file = os.path.join(src_test_path, file_name)
#
#         if os.path.isfile(src_file):  # Проверяем, что это файл, а не папка
#             shutil.copy2(src_file, dst_file)
#
# print("Копирование завершено!")

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.ImageFolder(root="data/train", transform=transform)
test_dataset = TensorDataset(
    torch.stack(
        [transform(Image.open(img).convert("RGB")) for img in sorted(glob.glob("data/test/*"))]
    ))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

In [4]:
def save_data(train_loader, test_loader, filename="dataset.h5"):

    with h5py.File(filename, "w") as f:
        train_group = f.create_group("train")
        test_group = f.create_group("test")
        train_images = train_group.create_dataset("images", (len(train_loader.dataset), 3, 224, 224), dtype="float32")
        train_labels = train_group.create_dataset("labels", (len(train_loader.dataset),), dtype="int")

        idx = 0
        for batch_images, batch_labels in train_loader:
            batch_size = batch_images.shape[0]
            train_images[idx:idx + batch_size] = batch_images.numpy()
            train_labels[idx:idx + batch_size] = batch_labels.numpy()
            idx += batch_size

        test_images = test_group.create_dataset("images", (len(test_loader.dataset), 3, 224, 224), dtype="float32")

        idx = 0
        for batch_images in test_loader:
            batch_size = len(batch_images[0])
            test_images[idx:idx + batch_size] = batch_images[0].numpy()
            idx += batch_size


In [5]:
def slice_dataloader(data, n):
    is_shuffled = isinstance(data.sampler, torch.utils.data.RandomSampler)
    if len(data.dataset[0]) > 1:
        all_data = []
        for images, labels in data:
            for img, label in zip(images, labels):
                all_data.append((img, label))
                if len(all_data) >= n:
                    break
            if len(all_data) >= n:
                break

        subset_images = [item[0] for item in all_data]
        subset_labels = [item[1] for item in all_data]
        return DataLoader(
            TensorDataset(torch.stack(subset_images), torch.tensor(subset_labels)),
            shuffle=is_shuffled,
            num_workers=data.num_workers,
            pin_memory=data.pin_memory,
            drop_last=True)
    else:
        all_data = []
        for images in data:
            for img in images:
                all_data.append(img)
                if len(all_data) >= n:
                    break
            if len(all_data) >= n:
                break

        subset_images = [item for item in all_data]
        return DataLoader(
            TensorDataset(torch.stack(subset_images)),
            shuffle=is_shuffled,
            num_workers=data.num_workers,
            pin_memory=data.pin_memory,
            drop_last=True)

In [9]:
test = slice_dataloader(test_loader, 100)

RuntimeError: stack expects each tensor to be equal size, but got [32, 3, 224, 224] at entry 0 and [8, 3, 224, 224] at entry 31

In [12]:
train = slice_dataloader(train_loader, 1000)
test = slice_dataloader(test_loader, 1000)
save_data(train, test, "small_dataset.h5")

RuntimeError: stack expects each tensor to be equal size, but got [32, 3, 224, 224] at entry 0 and [8, 3, 224, 224] at entry 31

In [6]:
# # Если что-то надо добавить
# def update_data(new_images):
#     with h5py.File("dataset.h5", "a") as f:  # "a" = append (добавление)
#         f["train/images"].resize((f["train/images"].shape[0] + new_images.shape[0]), axis=0)
#         f["train/images"][-new_images.shape[0]:] = new_images

In [2]:
size = 1000
with h5py.File("dataset.h5", "r") as f:
    train_images = torch.tensor(f["train/images"][:size], dtype=torch.float32)
    train_labels = torch.tensor(f["train/labels"][:size], dtype=torch.int)
    test_images = torch.tensor(f["test/images"][:size], dtype=torch.float32)

train_loader = DataLoader(TensorDataset(train_images, train_labels), batch_size=32, shuffle=True, )
test_loader = DataLoader(TensorDataset(test_images), batch_size=32, shuffle=False)

In [ ]:
img, label = next(iter(train_loader))
img = img[0]

img = img.permute(1, 2, 0).сpu().numpy()
img = img * 0.5 + 0.5

plt.imshow(img)
plt.axis("off")
plt.pause(0.001)
plt.show()

In [ ]:
img, label = next(iter(train_loader))
img = img[0]  # Берём первое изображение из батча

# Разнормализация (еслиЫ было transforms.Normalize(mean=[0.5], std=[0.5]))
img = img.permute(1, 2, 0).cpu().numpy()
img = ((img * 0.5 + 0.5)*255).astype(np.uint8)

image = Image.fromarray(img)
image.show()

In [ ]:
def show_image(loader):
    img, label = loader.dataset[0]
    img = img.numpy().transpose((1, 2, 0))
    img = img * 0.5 + 0.5
    plt.imshow(img)
    plt.title(loader.dataset.classes[label])
    plt.show()


show_image(train_loader)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

conv1_w = torch.randn((16, 3, 3, 3), requires_grad=True, device=device)  # 16 фильтров 3x3
conv2_w = torch.randn((32, 16, 3, 3), requires_grad=True, device=device) # 32 фильтра 3x3
fc1_w = torch.randn((32 * 8 * 8, 128), requires_grad=True, device=device) # Полносвязный слой
fc2_w = torch.randn((128, 10), requires_grad=True, device=device) # Выходной слой

params = [conv1_w, conv2_w, fc1_w, fc2_w]
optimizer = torch.optim.Adam(params, lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    correct, total, running_loss = 0, 0, 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        x = F.conv2d(images, conv1_w, stride=1, padding=1)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)

        x = F.conv2d(x, conv2_w, stride=1, padding=1)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)

        x = x.view(-1, 32 * 8 * 8)
        x = F.relu(x @ fc1_w)
        outputs = x @ fc2_w

        loss = F.cross_entropy(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item()

    train_acc = 100 * correct / total
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            x = F.conv2d(images, conv1_w, stride=1, padding=1)
            x = F.relu(x)
            x = F.max_pool2d(x, 2, 2)

            x = F.conv2d(x, conv2_w, stride=1, padding=1)
            x = F.relu(x)
            x = F.max_pool2d(x, 2, 2)

            x = x.view(-1, 32 * 8 * 8)
            x = F.relu(x @ fc1_w)
            outputs = x @ fc2_w

            # Accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100 * correct / total
    print(f"Test Accuracy: {test_acc:.2f}%\n")